### Simple MLP Regression

The y variable is a vector

In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load the data
data = pd.read_csv('county-year-agg.csv')

# replacing County with numeric value
data['County'].replace(['Accomack', 'Fauquier', 'Greensville', 'Hanover', 'Rockingham', 'Wise'],
                        [1,2,3,4,5,6], inplace=True)

data

## We only have 90 rows- likely need to bootstrap or synthetically create more data 

# scale variables!! (Standard scalar) 

,County,Year,Income,DSCI,P1,P2,P3,P4,P5,P6,...,VACO2,Agriculture,Aquaculture,Barren,Developed,Forest,Grassland,Open_Water,Shrubland,Wetlands
0,1,2008,32179,97.962264,34.02,34.84,35.16,35.66,40.25,41.26,...,116.763209,0.328149,0.0,0.012280,0.062278,0.032290,0.005097,0.048362,0.009128,0.502416
1,1,2009,32962,29.538462,44.98,41.86,44.18,43.11,40.44,40.58,...,105.907699,0.316069,0.0,0.019432,0.062453,0.026081,0.007146,0.050323,0.011391,0.507106
2,1,2010,33744,53.788462,56.54,60.19,62.93,60.25,59.17,55.46,...,111.886560,0.321777,0.0,0.098342,0.023141,0.302307,0.004368,0.043351,0.000000,0.206714
3,1,2011,33794,70.000000,38.80,36.37,32.59,33.49,32.24,37.00,...,102.519400,0.260725,0.0,0.023123,0.069094,0.061102,0.011748,0.048192,0.017834,0.508182
4,1,2012,35201,80.788462,40.11,42.78,41.63,43.26,45.27,42.18,...,98.562584,0.270210,0.0,0.026444,0.066762,0.051508,0.003030,0.046773,0.019485,0.515788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,6,2018,32442,8.115385,49.66,57.83,58.76,54.89,53.43,55.99,...,110.056786,0.011128,0.0,0.075743,0.077863,0.754591,0.073775,0.001891,0.004875,0.000134
86,6,2019,33793,33.905660,65.92,66.00,64.75,66.11,64.19,65.37,...,106.794626,0.020255,0.0,0.052630,0.083228,0.789954,0.040635,0.001948,0.011295,0.000057
87,6,2020,37115,5.538462,56.40,55.17,58.55,62.13,62.35,58.68,...,98.175849,0.020605,0.0,0.054942,0.083210,0.791840,0.035560,0.002085,0.011672,0.000086
88,6,2021,40444,10.057692,61.97,58.44,58.87,52.78,50.53,50.33,...,97.953403,0.015111,0.0,0.030682,0.091510,0.797472,0.042593,0.002082,0.020395,0.000155


In [4]:
# sort data by year before splitting into test/train
dfsort = data.sort_values(by=['Year'])

# leave out some counties- take 80pct of counties for train and rest for test?


# save input parameters to np.array
# Try again without hard coding the numbers - take an 80% fraction or so
Xtrain = dfsort.iloc[:72, :-9].to_numpy() #2008-2019
Xtest = dfsort.iloc[-18:-6, :-9].to_numpy() #2020-2022 

# save output variables to np.array
ytrain = dfsort.iloc[:72, -9:].to_numpy() #2008-2019
ytest = dfsort.iloc[-18:-6, -9:].to_numpy() #2020-2022 (remove 2022 bc it has NA values)
# maybe fill in NA's instead of removing those rows entirely. (interpolation function (BEFORE SCALING))

print(Xtrain)
print('----')
print(ytrain)

[[1.00000000e+00 2.00800000e+03 3.21790000e+04 ... 5.84000000e+01
  5.85000000e+01 1.16763209e+02]
 [3.00000000e+00 2.00800000e+03 2.58950000e+04 ... 5.93000000e+01
  5.92000000e+01 1.16763209e+02]
 [4.00000000e+00 2.00800000e+03 4.54120000e+04 ... 5.78000000e+01
  5.78000000e+01 1.16763209e+02]
 ...
 [2.00000000e+00 2.01900000e+03 7.07340000e+04 ... 5.65000000e+01
  5.66000000e+01 1.06794626e+02]
 [6.00000000e+00 2.01900000e+03 3.37930000e+04 ... 5.45000000e+01
  5.48000000e+01 1.06794626e+02]
 [4.00000000e+00 2.01900000e+03 6.09760000e+04 ... 5.90000000e+01
  5.92000000e+01 1.06794626e+02]]
----
[[3.28149486e-01 0.00000000e+00 1.22796590e-02 6.22778866e-02
  3.22901614e-02 5.09701475e-03 4.83618740e-02 9.12813267e-03
  5.02415785e-01]
 [1.50239898e-01 0.00000000e+00 3.70208192e-03 6.69430544e-02
  5.26131172e-01 4.04770323e-02 4.11537570e-03 5.45184835e-02
  1.53872902e-01]
 [1.83223246e-01 0.00000000e+00 2.97239230e-03 1.09551898e-01
  5.26886040e-01 5.83622448e-02 4.74336636e-03 4.

#### Fit the Model

In [26]:
# Fit the simple model
# HAVE CELL TO DEFINE HYPERPARAMETERS
# try setting and adding activation function (logistic / softmax / tanh)
# try to make it sum to number
# floor values (np.clip())

regr = MLPRegressor(hidden_layer_sizes=(100,),
                    random_state=1, 
                    activation='logistic',
                    learning_rate_init=0.01, 
                    learning_rate='adaptive',
                    max_iter=500000).fit(Xtrain, ytrain)

#### Make Predictions

In [27]:
# Make predictions for test data

regr.predict(Xtest[:1])

array([[ 0.1199204 , -0.13179691,  0.02108727, -0.04273388,  0.58007564,
         0.01936617,  0.05736348,  0.05967577,  0.01798712]])

In [28]:
ytest[:1]

array([[0.20079547, 0.        , 0.00219844, 0.07326492, 0.50980039,
        0.19144288, 0.00392987, 0.00466162, 0.0139064 ]])

In [41]:
# Make this all a function for each row!! This makes is like softmax
floored = np.clip(regr.predict(Xtest[:1]), 0,1)
sums = np.sum(floored)
floored/sums

array([[0.13697739, 0.        , 0.02408664, 0.        , 0.66258326,
        0.02212074, 0.06552263, 0.06816381, 0.02054554]])

In [ ]:
# compute mse instead of r^2 (Actual-predicted etc.)

In [ ]:
# could compare mse and r^2 for each column to see what land use it's best at predicting

#### Calc R-squared

In [29]:
# Calculate R-squared

regr.score(Xtest, ytest)

-105795.19850023062

In [25]:
regr.predict(Xtrain[:1])

array([[ 0.76157695, -0.51081324,  0.22482593, -0.09702468,  0.23622052,
         0.16578786,  0.81666421,  0.40803044,  0.01013608]])

In [20]:
regr.score(Xtrain, ytrain)

-2975432.5424045087

In [9]:
# 